In [18]:
from pprint import pprint

import sympy as sp
from sympy import pi, cos, sin, sqrt, atan2, acos, simplify 
from sympy.matrices import Matrix, eye
from mpmath import degrees, radians

In [19]:
sp.init_printing(use_latex='mathjax')

In [20]:
def column(spatial_components, homogeneous=False):
    '''
    Convenience function that ensures consistent use of appropriate column-aligned sympy vectors.

    :param spatial_components: ordered iterable of objects compatible with sympy matrices
    :param homogeneous: bool; generate homogeneous coordinate vector by adding a 4th row containing a 1
    '''
    column_matrix = [[item] for item in spatial_components]
    if homogeneous:
        column_matrix.append([1])
    return Matrix(column_matrix)


def sym_rot_x(q):
    return Matrix([
        [ 1,      0,       0],
        [ 0, cos(q), -sin(q)],
        [ 0, sin(q),  cos(q)]])

def sym_rot_y(q):
    return Matrix([
        [  cos(q), 0, sin(q)],
        [       0, 1,      0],
        [ -sin(q), 0, cos(q)]])

def sym_rot_z(q):
    return Matrix([
        [ cos(q), -sin(q), 0],
        [ sin(q),  cos(q), 0],
        [      0,       0, 1]])


def compose_sym_rotations(rotations):
    '''
    Compose a sympy rotation matrix corresponding to the sequence of rotations given.
    rotations: ordered iterable (of arbitrary length), which contains iterables of form (axis, radian_angle),
        where axis is an element of {'x', 'y', 'z'}.
    '''
    transform_about = {
        'x': sym_rot_x,
        'y': sym_rot_y,
        'z': sym_rot_z,
    }

    iteratively_composed_matrix = eye(3)

    for rotation in rotations:
        rotation_axis, rotation_angle = rotation[0], rotation[1]
        new_transform = transform_about[rotation_axis](rotation_angle)
        iteratively_composed_matrix = new_transform * iteratively_composed_matrix

    return iteratively_composed_matrix

In [87]:
sym_rot_x(q)

⎡1    0        0   ⎤
⎢                  ⎥
⎢0  cos(q)  -sin(q)⎥
⎢                  ⎥
⎣0  sin(q)  cos(q) ⎦

In [88]:
sym_rot_y(q)

⎡cos(q)   0  sin(q)⎤
⎢                  ⎥
⎢   0     1    0   ⎥
⎢                  ⎥
⎣-sin(q)  0  cos(q)⎦

In [89]:
sym_rot_z(q)

⎡cos(q)  -sin(q)  0⎤
⎢                  ⎥
⎢sin(q)  cos(q)   0⎥
⎢                  ⎥
⎣  0        0     1⎦

In [21]:
def symbolic_homogeneous_transform(rotation_list, translation):
    '''
    :param rotation_list: rotation sequence as specified by compose_sym_rotations function
    :param translation: iterable of cartesian coords of cumulative translation (x, y, z)
    :return: 4x4 sympy Matrix representing dh transform
    '''
    rot_matrix = compose_sym_rotations(rotation_list)
    return rot_matrix.col_join(Matrix([[0, 0, 0]])).row_join(
        column(translation, homogeneous=True))

In [26]:
# Elementary basis vectors x, y, z (non-homogeneous representation)
e_x = column((1, 0, 0))
e_y = column((0, 1, 0))
e_z = column((0, 0, 1))
# Homogeneous elementary basis vectors x, y, z
e_x_h = column((1, 0, 0), homogeneous=True)
e_y_h = column((0, 1, 0), homogeneous=True)
e_z_h = column((0, 0, 1), homogeneous=True)

(e_x, e_y, e_z), (e_x_h, e_y_h, e_z_h)

⎛                 ⎛⎡1⎤  ⎡0⎤  ⎡0⎤⎞⎞
⎜⎛⎡1⎤  ⎡0⎤  ⎡0⎤⎞  ⎜⎢ ⎥  ⎢ ⎥  ⎢ ⎥⎟⎟
⎜⎜⎢ ⎥  ⎢ ⎥  ⎢ ⎥⎟  ⎜⎢0⎥  ⎢1⎥  ⎢0⎥⎟⎟
⎜⎜⎢0⎥, ⎢1⎥, ⎢0⎥⎟, ⎜⎢ ⎥, ⎢ ⎥, ⎢ ⎥⎟⎟
⎜⎜⎢ ⎥  ⎢ ⎥  ⎢ ⎥⎟  ⎜⎢0⎥  ⎢0⎥  ⎢1⎥⎟⎟
⎜⎝⎣0⎦  ⎣0⎦  ⎣1⎦⎠  ⎜⎢ ⎥  ⎢ ⎥  ⎢ ⎥⎟⎟
⎝                 ⎝⎣1⎦  ⎣1⎦  ⎣1⎦⎠⎠

In [23]:
#######################################
# Create symbols
# symbols for DH parameters
q1, q2, q3, q4, q5, q6, q7 = sp.symbols('q1:8')  # theta angles
d1, d2, d3, d4, d5, d6, d7 = sp.symbols('d1:8')
a0, a1, a2, a3, a4, a5, a6 = sp.symbols('a0:7')
alpha0, alpha1, alpha2, alpha3, alpha4, alpha5, alpha6 = sp.symbols('alpha0:7')

# symbols for end effector target coordinates in base_link reference frame
x_target, y_target, z_target = sp.symbols('x_target y_target z_target')

# Create symbols for wrist center location in base frame
z_wrist, y_wrist, z_wrist = sp.symbols('wrist wrist z_wrist')

# Create Modified DH parameters
# Define the symbol dictionary which contains lookup values for the system
# ATTENTION! Note that joints 4-6 share a common origin, actually located at the wrist center,
# at the intersection of their Z axes.
dh_parameters = {alpha0: 0,              a0: 0,      d1: 0.75,
                 alpha1: -sp.pi / 2,     a1: 0.35,   d2: 0,      q2: q2 - sp.pi / 2,
                 alpha2: 0,              a2: 1.25,   d3: 0,
                 alpha3: -sp.pi / 2,     a3: -0.054, d4: 1.50,
                 alpha4: sp.pi / 2,      a4: 0,      d5: 0,
                 alpha5: -sp.pi / 2,     a5: 0,      d6: 0,
                 alpha6: 0,              a6: 0,      d7: 0.303,  q7: 0
                 }

#### Define Modified DH Transformation matrix

In [24]:
# Invoke generalized dh parameter symbols:
# alpha, a, d, q = twist_angle, link_length, joint_angle, link_offset
a, q, d, alpha = sp.symbols('a, q, d, alpha')

def single_dh_transform_step(alpha, a, d, q):
    '''Composes the Denavit-Hartenberg homogeneous transform matrix for the
    given values of alpha, a, d, and q.
    
    This matrix is produced by creating two precursors, each representing one 
    rotational and one translational component of the DH transform step, then
    composes them into a single transform and simplifies the result.
    '''
    sub_transform_x = symbolic_homogeneous_transform([('x', alpha)], (a, 0, 0))
    sub_transform_z = symbolic_homogeneous_transform([('z', q)], (0, 0, d))
    return sp.simplify(sub_transform_x * sub_transform_z)

generic_dh_tf_matrix = single_dh_transform_step(a, q, d, alpha)

pprint(generic_dh_tf_matrix)
generic_dh_tf_matrix

Matrix([
[       cos(alpha),       -sin(alpha),       0,         q],
[sin(alpha)*cos(a), cos(a)*cos(alpha), -sin(a), -d*sin(a)],
[sin(a)*sin(alpha), sin(a)*cos(alpha),  cos(a),  d*cos(a)],
[                0,                 0,       0,         1]])


⎡   cos(α)         -sin(α)        0         q    ⎤
⎢                                                ⎥
⎢sin(α)⋅cos(a)  cos(a)⋅cos(α)  -sin(a)  -d⋅sin(a)⎥
⎢                                                ⎥
⎢sin(a)⋅sin(α)  sin(a)⋅cos(α)  cos(a)   d⋅cos(a) ⎥
⎢                                                ⎥
⎣      0              0           0         1    ⎦

In [28]:
# Create individual transformation matrices
T0_1 = single_dh_transform_step(alpha0, a0, d1, q1).subs(dh_parameters)
T1_2 = single_dh_transform_step(alpha1, a1, d2, q2).subs(dh_parameters)
T2_3 = single_dh_transform_step(alpha2, a2, d3, q3).subs(dh_parameters)
T3_4 = single_dh_transform_step(alpha3, a3, d4, q4).subs(dh_parameters)
T4_5 = single_dh_transform_step(alpha4, a4, d5, q5).subs(dh_parameters)
T5_6 = single_dh_transform_step(alpha5, a5, d6, q6).subs(dh_parameters)
T6_7 = single_dh_transform_step(alpha6, a6, d7, q7).subs(dh_parameters)

In [29]:
T0_1

⎡cos(q₁)  -sin(q₁)  0   0  ⎤
⎢                          ⎥
⎢sin(q₁)  cos(q₁)   0   0  ⎥
⎢                          ⎥
⎢   0        0      1  0.75⎥
⎢                          ⎥
⎣   0        0      0   1  ⎦

In [30]:
T1_2

⎡sin(q₂)  cos(q₂)   0  0.35⎤
⎢                          ⎥
⎢   0        0      1   0  ⎥
⎢                          ⎥
⎢cos(q₂)  -sin(q₂)  0   0  ⎥
⎢                          ⎥
⎣   0        0      0   1  ⎦

In [31]:
T2_3

⎡cos(q₃)  -sin(q₃)  0  1.25⎤
⎢                          ⎥
⎢sin(q₃)  cos(q₃)   0   0  ⎥
⎢                          ⎥
⎢   0        0      1   0  ⎥
⎢                          ⎥
⎣   0        0      0   1  ⎦

In [32]:
T3_4

⎡cos(q₄)   -sin(q₄)  0  -0.054⎤
⎢                             ⎥
⎢   0         0      1   1.5  ⎥
⎢                             ⎥
⎢-sin(q₄)  -cos(q₄)  0    0   ⎥
⎢                             ⎥
⎣   0         0      0    1   ⎦

In [33]:
T4_5

⎡cos(q₅)  -sin(q₅)  0   0⎤
⎢                        ⎥
⎢   0        0      -1  0⎥
⎢                        ⎥
⎢sin(q₅)  cos(q₅)   0   0⎥
⎢                        ⎥
⎣   0        0      0   1⎦

In [34]:
T5_6

⎡cos(q₆)   -sin(q₆)  0  0⎤
⎢                        ⎥
⎢   0         0      1  0⎥
⎢                        ⎥
⎢-sin(q₆)  -cos(q₆)  0  0⎥
⎢                        ⎥
⎣   0         0      0  1⎦

In [35]:
T6_7

⎡1  0  0    0  ⎤
⎢              ⎥
⎢0  1  0    0  ⎥
⎢              ⎥
⎢0  0  1  0.303⎥
⎢              ⎥
⎣0  0  0    1  ⎦

In [51]:
T0_EE = sp.simplify(T0_1 * T1_2 * T2_3 * T3_4 * T4_5 * T5_6 * T6_7)

In [50]:
T0_EE

⎡((sin(q₁)⋅sin(q₄) + sin(q₂ + q₃)⋅cos(q₁)⋅cos(q₄))⋅cos(q₅) + sin(q₅)⋅cos(q₁)⋅c
⎢                                                                             
⎢((sin(q₁)⋅sin(q₂ + q₃)⋅cos(q₄) - sin(q₄)⋅cos(q₁))⋅cos(q₅) + sin(q₁)⋅sin(q₅)⋅c
⎢                                                                             
⎢                                -(sin(q₅)⋅sin(q₂ + q₃) - cos(q₄)⋅cos(q₅)⋅cos(
⎢                                                                             
⎣                                                                             

os(q₂ + q₃))⋅cos(q₆) - (-sin(q₁)⋅cos(q₄) + sin(q₄)⋅sin(q₂ + q₃)⋅cos(q₁))⋅sin(q
                                                                              
os(q₂ + q₃))⋅cos(q₆) - (sin(q₁)⋅sin(q₄)⋅sin(q₂ + q₃) + cos(q₁)⋅cos(q₄))⋅sin(q₆
                                                                              
q₂ + q₃))⋅cos(q₆) - sin(q₄)⋅sin(q₆)⋅cos(q₂ + q₃)                              
                                                   

In [52]:
R0_1 = T0_1[0:3, 0:3]
R1_2 = T1_2[0:3, 0:3]
R2_3 = T2_3[0:3, 0:3]
R3_4 = T3_4[0:3, 0:3]
R4_5 = T4_5[0:3, 0:3]
R5_6 = T5_6[0:3, 0:3]
R6_7 = T6_7[0:3, 0:3]

In [53]:
R0_EE = sp.simplify(R0_1 * R1_2 * R2_3 * R3_4 * R4_5 * R5_6 * R6_7)

In [54]:
R0_EE

⎡((sin(q₁)⋅sin(q₄) + sin(q₂ + q₃)⋅cos(q₁)⋅cos(q₄))⋅cos(q₅) + sin(q₅)⋅cos(q₁)⋅c
⎢                                                                             
⎢((sin(q₁)⋅sin(q₂ + q₃)⋅cos(q₄) - sin(q₄)⋅cos(q₁))⋅cos(q₅) + sin(q₁)⋅sin(q₅)⋅c
⎢                                                                             
⎣                                -(sin(q₅)⋅sin(q₂ + q₃) - cos(q₄)⋅cos(q₅)⋅cos(

os(q₂ + q₃))⋅cos(q₆) - (-sin(q₁)⋅cos(q₄) + sin(q₄)⋅sin(q₂ + q₃)⋅cos(q₁))⋅sin(q
                                                                              
os(q₂ + q₃))⋅cos(q₆) - (sin(q₁)⋅sin(q₄)⋅sin(q₂ + q₃) + cos(q₁)⋅cos(q₄))⋅sin(q₆
                                                                              
q₂ + q₃))⋅cos(q₆) - sin(q₄)⋅sin(q₆)⋅cos(q₂ + q₃)                              

₆)  -((sin(q₁)⋅sin(q₄) + sin(q₂ + q₃)⋅cos(q₁)⋅cos(q₄))⋅cos(q₅) + sin(q₅)⋅cos(q
                                                                              
)   -((sin(q₁)⋅sin(q₂ + q₃)⋅cos(q₄) - sin(q₄)⋅cos(

In [55]:
R0_EE_from_dhtf = rotation_sub_matrix(T0_EE)

In [18]:
R0_EE == R0_EE_from_dhtf

True

### The following cells show the outcome of using the correction matrix from the walkthrough
using the mpmath radians function rather than sympy's pi object introduces a miniscule amount of error, but why introduce any numerical error at all when the desired result is encoded PERFECTLY by a clean, symbolic solution?

In [57]:
# Conversion Factors - Note that this is using the symbolic definition of pi from SymPy
rtd = 180 / sp.pi  # radians to degrees
dtr = sp.pi / 180  # degrees to radians

r, p, y = sp.symbols('r p y')
R_corr_sym = sym_rot_z(y) * sym_rot_y(p)


# Walkthrough method
R_corr_a = sym_rot_z(y).subs(y, radians(180)) * sym_rot_y(p).subs(p, radians(-90))
# Pure symbolic method
R_corr_b = R_corr_sym.subs({y: dtr * 180, p: dtr * -90})

R_corr_sym

⎡cos(p)⋅cos(y)  -sin(y)  sin(p)⋅cos(y)⎤
⎢                                     ⎥
⎢sin(y)⋅cos(p)  cos(y)   sin(p)⋅sin(y)⎥
⎢                                     ⎥
⎣   -sin(p)        0        cos(p)    ⎦

In [59]:
R_corr_a

⎡-6.12323399573677e-17  -1.22464679914735e-16           1.0         ⎤
⎢                                                                   ⎥
⎢7.49879891330929e-33           -1.0           -1.22464679914735e-16⎥
⎢                                                                   ⎥
⎣         1.0                     0            6.12323399573677e-17 ⎦

In [74]:
R_corr = R_corr_b
R_corr_b

⎡0  0   1⎤
⎢        ⎥
⎢0  -1  0⎥
⎢        ⎥
⎣1  0   0⎦

In [75]:
R_roll = sym_rot_x(r)
R_pitch = sym_rot_y(p)
R_yaw = sym_rot_z(y)
R_roll, R_pitch, R_yaw

⎛⎡1    0        0   ⎤  ⎡cos(p)   0  sin(p)⎤  ⎡cos(y)  -sin(y)  0⎤⎞
⎜⎢                  ⎥  ⎢                  ⎥  ⎢                  ⎥⎟
⎜⎢0  cos(r)  -sin(r)⎥, ⎢   0     1    0   ⎥, ⎢sin(y)  cos(y)   0⎥⎟
⎜⎢                  ⎥  ⎢                  ⎥  ⎢                  ⎥⎟
⎝⎣0  sin(r)  cos(r) ⎦  ⎣-sin(p)  0  cos(p)⎦  ⎣  0        0     1⎦⎠

In [76]:
ree, ry, rp, rr, rc = sp.symbols("R_EE, R_yaw, R_pitch, R_roll, R_corr")

In [92]:
# sp.Equivalent(ry, R_yaw)

In [93]:
# Equivalent(rp, R_pitch)

In [94]:
# sp.Equivalent(rr, R_roll)

In [95]:
# sp.Equivalent(rc, R_corr)

In [96]:
R_rpy = R_yaw * R_pitch * R_roll
R_rpy

⎡cos(p)⋅cos(y)  sin(p)⋅sin(r)⋅cos(y) - sin(y)⋅cos(r)  sin(p)⋅cos(r)⋅cos(y) + s
⎢                                                                             
⎢sin(y)⋅cos(p)  sin(p)⋅sin(r)⋅sin(y) + cos(r)⋅cos(y)  sin(p)⋅sin(y)⋅cos(r) - s
⎢                                                                             
⎣   -sin(p)                sin(r)⋅cos(p)                         cos(p)⋅cos(r)

in(r)⋅sin(y)⎤
            ⎥
in(r)⋅cos(y)⎥
            ⎥
            ⎦

In [97]:
R_EE = R_yaw * R_pitch * R_roll * R_corr
R_EE

⎡sin(p)⋅cos(r)⋅cos(y) + sin(r)⋅sin(y)  -sin(p)⋅sin(r)⋅cos(y) + sin(y)⋅cos(r)  
⎢                                                                             
⎢sin(p)⋅sin(y)⋅cos(r) - sin(r)⋅cos(y)  -sin(p)⋅sin(r)⋅sin(y) - cos(r)⋅cos(y)  
⎢                                                                             
⎣           cos(p)⋅cos(r)                         -sin(r)⋅cos(p)              

cos(p)⋅cos(y)⎤
             ⎥
sin(y)⋅cos(p)⎥
             ⎥
   -sin(p)   ⎦

In [98]:
R0_3 = sp.simplify(R0_1 * R1_2 * R2_3)
R0_3

⎡sin(q₂ + q₃)⋅cos(q₁)  cos(q₁)⋅cos(q₂ + q₃)  -sin(q₁)⎤
⎢                                                    ⎥
⎢sin(q₁)⋅sin(q₂ + q₃)  sin(q₁)⋅cos(q₂ + q₃)  cos(q₁) ⎥
⎢                                                    ⎥
⎣    cos(q₂ + q₃)         -sin(q₂ + q₃)         0    ⎦

In [ ]:
R0_EE = sp.simplify(R0_3 * R3_4 * R4_5 * R5_6 * R6_7)

In [102]:
R0_EE

⎡((sin(q₁)⋅sin(q₄) + sin(q₂ + q₃)⋅cos(q₁)⋅cos(q₄))⋅cos(q₅) + sin(q₅)⋅cos(q₁)⋅c
⎢                                                                             
⎢((sin(q₁)⋅sin(q₂ + q₃)⋅cos(q₄) - sin(q₄)⋅cos(q₁))⋅cos(q₅) + sin(q₁)⋅sin(q₅)⋅c
⎢                                                                             
⎣                               -(sin(q₅)⋅sin(q₂ + q₃) - cos(q₄)⋅cos(q₅)⋅cos(q

os(q₂ + q₃))⋅cos(q₆) + (sin(q₁)⋅cos(q₄) - sin(q₄)⋅sin(q₂ + q₃)⋅cos(q₁))⋅sin(q₆
                                                                              
os(q₂ + q₃))⋅cos(q₆) - (sin(q₁)⋅sin(q₄)⋅sin(q₂ + q₃) + cos(q₁)⋅cos(q₄))⋅sin(q₆
                                                                              
₂ + q₃))⋅cos(q₆) - sin(q₄)⋅sin(q₆)⋅cos(q₂ + q₃)                               

)  -((sin(q₁)⋅sin(q₄) + sin(q₂ + q₃)⋅cos(q₁)⋅cos(q₄))⋅cos(q₅) + sin(q₅)⋅cos(q₁
                                                                              
)  -((sin(q₁)⋅sin(q₂ + q₃)⋅cos(q₄) - sin(q₄)⋅cos(q

In [103]:
R0_3_inverse = sp.simplify(R0_3.inv('LU'))

In [104]:
R0_3_inverse

⎡sin(q₂ + q₃)⋅cos(q₁)  sin(q₁)⋅sin(q₂ + q₃)  cos(q₂ + q₃) ⎤
⎢                                                         ⎥
⎢cos(q₁)⋅cos(q₂ + q₃)  sin(q₁)⋅cos(q₂ + q₃)  -sin(q₂ + q₃)⎥
⎢                                                         ⎥
⎣      -sin(q₁)              cos(q₁)               0      ⎦

And to verify that this result is truly what we were seeking, we can multiply it by R0_3 to check that the result is an identity matrix:

In [106]:
(simplify(R0_3 * R0_3_inverse) == eye(3)) == True

True

This result is totally independent of the pose in question; so long as we can calculate q1, q2, and q3 values, we can determine the numerical-valued form of R0_3_inverse with nothing more than a substitution.

In [109]:
R3_6 = sp.simplify(R0_3_inverse * R0_EE)

In [110]:
R3_6

⎡                                                                             
⎢-sin(q₄)⋅sin(q₆) + cos(q₄)⋅cos(q₅)⋅cos(q₆)  -sin(q₄)⋅cos(q₆) - sin(q₆)⋅cos(q₄
⎢                                                                             
⎢                                                                             
⎢             sin(q₅)⋅cos(q₆)                             -sin(q₅)⋅sin(q₆)    
⎢                                                                             
⎣-sin(q₄)⋅cos(q₅)⋅cos(q₆) - sin(q₆)⋅cos(q₄)  sin(q₄)⋅sin(q₆)⋅cos(q₅) - cos(q₄)

           sin(q₄ - q₅)   sin(q₄ + q₅)⎤
)⋅cos(q₅)  ──────────── - ────────────⎥
                2              2      ⎥
                                      ⎥
                     cos(q₅)          ⎥
                                      ⎥
⋅cos(q₆)         sin(q₄)⋅sin(q₅)      ⎦

In [114]:
R3_6 = Matrix([
    [-sin(q4)*sin(q6) + cos(q4)*cos(q5)*cos(q6), -sin(q4)*cos(q6) - sin(q6)*cos(q4)*cos(q5), -cos(q4)*sin(q5)],
    [                           sin(q5)*cos(q6),                           -sin(q5)*sin(q6),          cos(q5)],
    [-sin(q4)*cos(q5)*cos(q6) - sin(q6)*cos(q4),  sin(q4)*sin(q6)*cos(q5) - cos(q4)*cos(q6),  sin(q4)*sin(q5)]])
R3_6

⎡-sin(q₄)⋅sin(q₆) + cos(q₄)⋅cos(q₅)⋅cos(q₆)  -sin(q₄)⋅cos(q₆) - sin(q₆)⋅cos(q₄
⎢                                                                             
⎢             sin(q₅)⋅cos(q₆)                             -sin(q₅)⋅sin(q₆)    
⎢                                                                             
⎣-sin(q₄)⋅cos(q₅)⋅cos(q₆) - sin(q₆)⋅cos(q₄)  sin(q₄)⋅sin(q₆)⋅cos(q₅) - cos(q₄)

)⋅cos(q₅)  -sin(q₅)⋅cos(q₄)⎤
                           ⎥
               cos(q₅)     ⎥
                           ⎥
⋅cos(q₆)   sin(q₄)⋅sin(q₅) ⎦

In [76]:
(0.5*(sin(q4 - q5) - sin(q4 + q5))).evalf(subs={q4: 1.1125, q5: 0.3*sp.pi})

-0.357926012846339

In [77]:
(-cos(q4)*sin(q5)).evalf(subs={q4: 1.1125, q5: 0.3*sp.pi})

-0.357926012846339

In [80]:
(-cos(q4)*sin(q5)).evalf(subs={q4: 1.1125, q5: 0.3*sp.pi}) == (0.5*(sin(q4 - q5) - sin(q4 + q5))).evalf(subs={q4: 1.1125, q5: 0.3*sp.pi})

True

In [115]:
r_11 = R3_6[0, 0]
r_12 = R3_6[0, 1]
r_13 = R3_6[0, 2]
r_21 = R3_6[1, 0]
r_22 = R3_6[1, 1]
r_23 = R3_6[1, 2]
r_31 = R3_6[2, 0]
r_32 = R3_6[2, 1]
r_33 = R3_6[2, 2]

The objective here is to come up with formulas for q4, q5, and q6 in terms of particular indexed elements of a 3x3 matrix.
The rotation matrix R3_6_edit, above, is defined in terms of DH parameters, which we are solving for, but the matrix we will draw values from
is the numerical matrix R3_6 = R0_3_inverse.dot(r_EE), where R0_3_inverse is determined by q1-q3, and r_EE is determined by the rpy euler angles from the pose.

In a sense, the above 'r_ij' variables can be considered placeholders for numerical values that will be mapped to joint angles through relationships this matrix encodes.

### Solving q4

In [116]:
r_33, r_13, r_33 / -r_13

⎛                                   sin(q₄)⎞
⎜sin(q₄)⋅sin(q₅), -sin(q₅)⋅cos(q₄), ───────⎟
⎝                                   cos(q₄)⎠

In [108]:
r_33 / -r_13

sin(q₄)
───────
cos(q₄)

### Solving q5

In [126]:
r_13, r_33, r_23

(-sin(q₅)⋅cos(q₄), sin(q₄)⋅sin(q₅), cos(q₅))

In [116]:
r_13 ** 2 + r_33 ** 2

   2        2          2        2    
sin (q₄)⋅sin (q₅) + sin (q₅)⋅cos (q₄)

In [125]:
sp.factor(r_13 ** 2 + r_33 ** 2), sp.simplify(r_13 ** 2 + r_33 ** 2)

⎛⎛   2          2    ⎞    2         2    ⎞
⎝⎝sin (q₄) + cos (q₄)⎠⋅sin (q₅), sin (q₅)⎠

In [117]:
sp.sqrt(sp.simplify(r_13 ** 2 + r_33 ** 2)) / r_23, sin(q5)/cos(q5)

⎛   __________         ⎞
⎜  ╱    2              ⎟
⎜╲╱  sin (q₅)   sin(q₅)⎟
⎜─────────────, ───────⎟
⎝   cos(q₅)     cos(q₅)⎠

In [141]:
q5 == sp.atan2(sp.sqrt(R3_6[0, 2] * R3_6[0, 2] + R3_6[2, 2] * R3_6[2, 2]), R3_6[1, 2])

False

### Solving q6

In [120]:
theta6 = atan2(-R3_6[1, 1], R3_6[1, 0])

In [123]:
r_22, r_21, -r_22 / r_21

⎛                                   sin(q₆)⎞
⎜-sin(q₅)⋅sin(q₆), sin(q₅)⋅cos(q₆), ───────⎟
⎝                                   cos(q₆)⎠

Working Result:
```
theta5 = atan2(sqrt(R3_6[0, 2] * R3_6[0, 2] + R3_6[2, 2] * R3_6[2, 2]), R3_6[1, 2]) # r_13**2 + r_33**2, r_23
theta4 = atan2(R3_6[2, 2], -R3_6[0, 2])
theta6 = atan2(-R3_6[1, 1], R3_6[1, 0])
```